# [모듈 2.0] 모델 빌딩하기 (No VPC 환경에서 실행하세요)



In [1]:
import tensorflow as tf
tf.__version__

'2.1.2'

In [2]:
pip install sagemaker --upgrade

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# (3) Use SageMaker local for local testing


In [3]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

`sagemaker.tensorflow` 클래스를 사용하여 SageMaker Python SDK의 Tensorflow Estimator 인스턴스를 생성합니다.
인자값으로 하이퍼파라메터와 다양한 설정들을 변경할 수 있습니다.

자세한 내용은 [documentation](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-tensorflow-estimator)을 확인하시기 바랍니다.

In [4]:
dataset_location = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-ap-northeast-2-057716757052/data/DEMO-cifar10'

## (5) DDP

In [6]:
from sagemaker.tensorflow import TensorFlow
pt_estimator = TensorFlow(
                        base_job_name='tf-cifar10-ddp',
                        source_dir='training_script',    
                        entry_point='cifar10_keras_sm_ddp_82.py',
                        role=role,
                        py_version='py37',
                        framework_version='2.3.1',
                        # For training with multinode distributed training, set this count. Example: 2
                        instance_count=1,
                        # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
                        instance_type= 'ml.p3.16xlarge',
                        sagemaker_session=sagemaker_session,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                             }
                                      }}
                        )
pt_estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2021-02-22 13:59:14 Starting - Starting the training job...
2021-02-22 13:59:36 Starting - Launching requested ML instancesProfilerReport-1614002353: InProgress
.........
2021-02-22 14:00:58 Starting - Preparing the instances for training............
2021-02-22 14:03:09 Downloading - Downloading input data
2021-02-22 14:03:09 Training - Downloading the training image...............
2021-02-22 14:05:41 Training - Training image download completed. Training in progress.2021-02-22 14:05:31.763971: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-02-22 14:05:31.768381: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-02-22 14:05:32.006951: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2021-02-22 14:05:32.101691: W tensorfl

UnexpectedStatusException: Error for Training job tf-cifar10-ddp-2021-02-22-13-59-13-709: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "mpirun --host algo-1 -np 8 --allow-run-as-root --tag-output --oversubscribe -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca btl_vader_single_copy_mechanism none -mca plm_rsh_num_concurrent 1 -x NCCL_SOCKET_IFNAME=eth0 -x LD_LIBRARY_PATH -x PATH -x SMDATAPARALLEL_USE_SINGLENODE=1 -x FI_PROVIDER=sockets -x RDMAV_FORK_SAFE=1 -x LD_PRELOAD=/usr/local/lib/python3.7/site-packages/gethostname.cpython-37m-x86_64-linux-gnu.so smddprun /usr/local/bin/python3.7 -m mpi4py cifar10_keras_sm_ddp_82.py --model_dir s3://sagemaker-ap-northeast-2-057716757052/tf-cifar10-ddp-2021-02-22-13-59-13-709/model"
[1,5]<stderr>:2021-02-22 14:05:36.988027: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler